In [12]:
#import libraries
from sklearn.datasets import load_boston # dataset
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
import statsmodels.stats.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
import statsmodels as sm
import statsmodels.formula.api as smf
from sklearn import linear_model

%matplotlib inline

In [13]:
# instantiate dataset and create DataFrame 
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
target = pd.DataFrame(boston.target, columns=["MEDV"])

In [14]:
# Basic exploratory analyses 
print("Rows,Columns:",df.shape)
df.describe()

Rows,Columns: (506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [15]:
# split dataset into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(df, target, test_size=0.30)


In [16]:
# Fitting using stats model
model_1 = smf.OLS(Y_train,X_train).fit()
prediction_1 = model_1.predict(X_train) # predict y_train values
prediction_2 = model_1.predict(X_test)# predict y_test values

In [17]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     557.1
Date:                Sun, 07 Jan 2018   Prob (F-statistic):          1.17e-220
Time:                        17:56:49   Log-Likelihood:                -1078.8
No. Observations:                 354   AIC:                             2184.
Df Residuals:                     341   BIC:                             2234.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0956      0.043     -2.236      0.026      -0.180      -0.011
ZN             0.0554      0.017      3.192      0.002       0.021       0.089
INDUS          0.0056      0.082      0.068      0.946      -0.155       0.166
CHAS           1.9569      1.172      1.670      0.096      -0.348       4.262
NOX           -2.8633      4.068     -0.704      0.482     -10.865       5.138
RM             5.4115      0.389     13.909      0.000       4.646       6.177
AGE            0.0126      0.018      0.710      0.478      -0.022       0.047
DIS           -0.8650      0.239     -3.622      0.000      -1.335      -0.395
RAD            0.1944      0.081      2.388      0.018       0.034       0.355
TAX           -0.0103      0.005     -2.192      0.029      -0.019      -0.001
PTRATIO       -0.2925      0.139     -2.104      0.036      -0.566      -0.019
B              0.0158      0.003      4.577      0.000       0.009       0.023
LSTAT         -0.4700      0.063     -7.461      0.000      -0.594      -0.346
==============================================================================
Omnibus:                      134.653   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              653.132
Skew:                           1.552   Prob(JB):                    1.49e-142
Kurtosis:                       8.886   Cond. No.                     8.28e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Looking at the Summary

In the summary we see some interesting results. The INDUS coefficient (proportion of non-retail business acres per town) is not nearly statistically significant at the 5% or 10% level. The same goes for the AGE and NOX variables.

There seems to be no autocorrelation (as expected as this is not a time-series problem).

Overall, the Adjusted R^2 score is great, but how about RMSE (root mean square error), lets have a loo

In [18]:
rmse_training = sqrt(mean_squared_error(Y_train, prediction_1))
print("RMSE for training data:", rmse_training)
rmse_test = sqrt(mean_squared_error(Y_test,prediction_2))
print("RMSE for test data:", rmse_test)

RMSE for training data: 5.095701134618519
RMSE for test data: 4.580486233041159



Hmm... The RMSE for the test data is quite low and very close to that of the training data. But can we do better?! Lets try by doing a correlation test to identify spurious variables. **A VIF Test.**

In [20]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_test.values, i) for i in range(X_test.shape[1])]
vif["features"] = X_test.columns
vif

,VIF Factor,features
0,2.093688,CRIM
1,3.286848,ZN
2,14.367665,INDUS
3,1.171446,CHAS
4,91.379192,NOX
5,80.456933,RM
6,20.601732,AGE
7,15.631933,DIS
8,18.107816,RAD
9,78.637935,TAX


There is no hard and fast rules on VIF factors, while a score above 10 is undesirable as it suggests very correlation that artificially inflates our R score, a subjective call must be taken depending on the problem. For example, while the TAX variable (full-value property-tax rate per 10,000 dollars) is highly correlated with the rest of the variables, it is too important of a variable to ignore, plus it is very significant at the 5% level.

NOX and INDUS on the other hand, we can try and remove and see if we get a better model. Let's re-create our training sets.

In [21]:

#create second training set
#df = df.drop(["NOX","INDUS"], axis = 1)
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(df, target, test_size=0.30)
print(df.shape)
print(X_test1.shape)

(506, 13)
(152, 13)


In [22]:
# fit second model
model_2 = smf.OLS(Y_train1,X_train1).fit()
prediction_3 = model_2.predict(X_train1)# predict y_test values
prediction_4 = model_2.predict(X_test1)# predict y_test values


In [23]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     615.7
Date:                Sun, 07 Jan 2018   Prob (F-statistic):          9.89e-228
Time:                        17:58:51   Log-Likelihood:                -1071.1
No. Observations:                 354   AIC:                             2168.
Df Residuals:                     341   BIC:                             2219.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0806      0.037     -2.159      0.032      -0.154      -0.007
ZN             0.0490      0.018      2.715      0.007       0.014       0.084
INDUS          0.0424      0.081      0.521      0.603      -0.118       0.202
CHAS           3.1340      1.066      2.940      0.004       1.037       5.231
NOX           -8.3528      4.109     -2.033      0.043     -16.435      -0.270
RM             6.2517      0.392     15.943      0.000       5.480       7.023
AGE            0.0075      0.017      0.430      0.668      -0.027       0.042
DIS           -0.9659      0.241     -4.004      0.000      -1.440      -0.491
RAD            0.1426      0.081      1.754      0.080      -0.017       0.303
TAX           -0.0080      0.005     -1.630      0.104      -0.018       0.002
PTRATIO       -0.3971      0.133     -2.977      0.003      -0.659      -0.135
B              0.0137      0.004      3.884      0.000       0.007       0.021
LSTAT         -0.4411      0.063     -6.968      0.000      -0.566      -0.317
==============================================================================
Omnibus:                      151.750   Durbin-Watson:                   1.852
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1226.002
Skew:                           1.585   Prob(JB):                    5.99e-267
Kurtosis:                      11.548   Cond. No.                     8.55e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.55e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
rmse_training_2 = sqrt(mean_squared_error(Y_train1, prediction_3))
print("RMSE for training data:", rmse_training)
rmse_test_2 = sqrt(mean_squared_error(Y_test1,prediction_4))
print("RMSE for test data:", rmse_test)

RMSE for training data: 5.095701134618519
RMSE for test data: 4.580486233041159


In [25]:
lm = linear_model.LinearRegression()
model_2 = lm.fit(X_train,Y_train)

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [26]:

rms = sqrt(mean_squared_error(Y_test, prediction_2))
print("RMSE_test:",rms)
rms_1 = sqrt(mean_squared_error(Y_train,prediction_1))
print("RMSE_train:",rms_1)

RMSE_test: 4.580486233041159
RMSE_train: 5.095701134618519


In [28]:
residual = Y_test.values-prediction_2
#print(residual)
for i in X_test:
    plt.scatter(X_test[i],residual)
    plt.xlim(0,)
    plt.xlabel(i)
    plt.ylabel("residuals")
    plt.show()


Exception: Data must be 1-dimensional

## Test for Heteroskedacticity (Breush-Pagan)

In [37]:
name = [ 'p-value']
test = smf.het_breuschpagan(model_1.resid, model_1.model.exog)
lzip(name, test)

AttributeError: module 'statsmodels.formula.api' has no attribute 'het_breuschpagan'